# Sage Combinat Widgets Adapter Tutorial

In [ ]:
# Get rid of long error messages for clarity.
from __future__ import print_function
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler
#%xmode Plain # in case you want the tracebacks, uncomment this line and comment above.

Suppose we want to play with a matrix, ie edit cells, add or drop rows or columns.

In [ ]:
%display latex
from sage.matrix.constructor import Matrix
m = Matrix(ZZ, 3, 4, range(12))
m

To be able to display this matrix as a widget, we need to write an *adapter* [NB: here, link to a doc that explains in more details what an adapter is]. As we want a *grid-like* representation for our matrix, we will base our adapter on *GridViewAdapter*:

In [ ]:
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    pass

Let's check what happens at this stage:

In [ ]:
from sage_combinat_widgets import GridViewWidget
ma = MyAdapter()
GridViewWidget(m,ma)

We need to tell the widget which cells are to be displayed in the grid view.

In [ ]:
#from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

In [ ]:
ma = MyAdapter()
GridViewWidget(m,ma)

We need to tell the widget which type are our matrix cells, and which would be a default cell content (for blank or empty cells).

In [ ]:
#from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0) # pas obligatoire pour

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

Let's try again

In [ ]:
ma = MyAdapter()
w = GridViewWidget(m,ma)
w

We can see the graphical display. Yet if we try to edit a cell, our matrix is unchanged:

In [ ]:
w.value

Now we will learn how to make our widget editable.

## Editing cells

In order to turn our widget into an editable matrix widget, we need a method to establish a new matrix value from a new dictionary of cells.

In [ ]:
from sage.matrix.constructor import matrix
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    constructorname = 'matrix'
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

    @classmethod
    def from_cells(cls, cells={}):
        """Compute a new matrix from a cells dictionary `cells`"""
        nrows = max(pos[0]+1 for pos in cells)
        ncols = max(pos[1]+1 for pos in cells)
        return cls.constructorname([[cells[(i,j)] for j in range(ncols)] for i in range(nrows)])

In [ ]:
#from sage_combinat_widgets import GridViewWidget
ma = MyAdapter()
w = GridViewWidget(m,ma)
w

In [ ]:
# Observe the value change after editing a cell. 
w.value

In [ ]:
from ipywidgets import HBox, Label
m_input = GridViewWidget(m)
@interact
def f(x = m_input):
    return HBox((Label('m mult. by 2 --->'), GridViewWidget(x*2)))

## Interactivity modes

Besides editing cells, we can define modes of interactivity, such as adding or removing cells ... 
For a matrix, we want to append/insert/remove rows or columns.
Here is the code for appending a row/column:

In [ ]:
from sage.matrix.constructor import matrix
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
class MyAdapter(GridViewAdapter):
    constructorname = 'matrix'
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

    @classmethod
    def from_cells(cls, cells={}):
        """Compute a new matrix from a cells dictionary `cells`"""
        nrows = max(pos[0]+1 for pos in cells)
        ncols = max(pos[1]+1 for pos in cells)
        return cls.constructorname([[cells[(i,j)] for j in range(ncols)] for i in range(nrows)])

    def append_row(self, obj, r):
        """Return a new matrix with one more row 'r'"""
        if not r or not hasattr(r, '__len__') or len(r) > obj.ncols():
            raise ValueError("Row size must be <= %d" % obj.ncols())
        if len(r) < obj.ncols():
            r = list(r) + [self.cellzero] * (obj.ncols() - len(r))
        return obj.stack(vector([self.display_to_cell(x) for x in r]))

    def append_column(self, obj, c):
        """Return a new matrix with one more column 'c'"""
        if not c or not hasattr(c, '__len__') or len(c) > obj.nrows():
            raise ValueError("Column size must be <= %d" % obj.nrows())
        if len(c) < obj.nrows():
            c = list(c) + [self.cellzero] * (obj.nrows() - len(c))
        return obj.augment(vector([self.display_to_cell(x) for x in c]))

In [ ]:
ma = MyAdapter()
w = GridViewWidget(m,ma)
w.append_row((12,13,14,15))
w

Finally, we can drive these interactivity modes with a pair of buttons: 

In [ ]:
from ipywidgets import HBox, VBox, Button
b_newrow = Button()
b_newrow.description = "Append row"
b_newcol = Button()
b_newcol.description = "Append column"

w = GridViewWidget(m,ma)
def on_button_clicked(button):
    w.append_row((w.value[-1][0]+4, w.value[-1][1]+4, w.value[-1][2]+4, w.value[-1][3]+4))
b_newrow.on_click(on_button_clicked)
def on_button_clicked(button):
    w.append_column((w.value[0][-1]*2, w.value[1][-1]*2, w.value[2][-1]*2))
b_newcol.on_click(on_button_clicked)

box = VBox((HBox((w, b_newcol)), b_newrow))
box

For more interactivity modes, please consult the generic adapter documentation.

For various examples of widgets, see the demo.